In [90]:
import pandas as pd

education = pd.read_csv('private_education.csv')
education['대도시외'] = education['중소도시'] + education['읍면지역']
education = education.drop(columns=['전체', '중소도시', '읍면지역'])
education.head()


,시점,서울,광역시,대도시외
0,2007,47410,54016,98973
1,2008,48258,55401,105435
2,2009,52502,51584,112173
3,2010,49119,49716,109882
4,2011,48018,47021,106226


In [91]:
education['시점'] = pd.to_datetime(education['시점'], format='%Y', errors='coerce')

# 각 지역별 데이터 숫자형으로 변환 (문자열에서 숫자로 변환)
for column in ['서울', '광역시', '대도시외']:
    education[column] = pd.to_numeric(education[column], errors='coerce')

education.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   시점      17 non-null     datetime64[ns]
 1   서울      17 non-null     int64         
 2   광역시     17 non-null     int64         
 3   대도시외    17 non-null     int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 676.0 bytes


In [92]:
# 월별 인덱스 생성
monthly_index = pd.date_range(start=education['시점'].min(), end=education['시점'].max(), freq='ME')

# 월별 데이터 프레임 생성
monthly_education = pd.DataFrame()

# 각 지역별 데이터를 추가
for column in ['서울', '광역시', '대도시외']:
    # 해당 지역의 연도별 데이터만 선택하고 인덱스를 연도 형태로 변환
    yearly_data = education.set_index('시점')[column]
    
    # 보간을 위한 리샘플링
    monthly_data = yearly_data.resample('ME').mean()  # 평균으로 리샘플링
    monthly_data = monthly_data.interpolate(method='linear')  # 선형 보간
    monthly_education[column] = monthly_data.astype(int)

# 시점 포맷 변환
monthly_education.reset_index(inplace=True)
monthly_education.rename(columns={'index': '시점'}, inplace=True)
monthly_education['시점'] = monthly_education['시점'].dt.strftime('%Y.%m')

monthly_education.head(12), monthly_education.shape

# 데이터 저장 (선택 사항)
monthly_education.to_csv('monthly_private_education.csv')


(         시점     서울    광역시    대도시외
 0   2007.01  47410  54016   98973
 1   2007.02  47480  54131   99511
 2   2007.03  47551  54246  100050
 3   2007.04  47622  54362  100588
 4   2007.05  47692  54477  101127
 5   2007.06  47763  54593  101665
 6   2007.07  47834  54708  102204
 7   2007.08  47904  54823  102742
 8   2007.09  47975  54939  103281
 9   2007.10  48046  55054  103819
 10  2007.11  48116  55170  104358
 11  2007.12  48187  55285  104896,
 (193, 4))